# Understanding Historical Climate Data on the Analytics Engine
This notebook is a walkthrough of how to utilize different kinds of historical climate data, including weather observations, reanalysis products, and model output available on the Analytics Engine. 

* Weather observations are inherently point locations, tied to a single station location, and represent the actual values of weather variables. Weather observations are highly localized weather information, and are limited by instrumentation constraints.
* Reanalysis products are reconstructions of the historical weather observation period. Like a climate model, reanalysis products have a complete set of atmospheric and surface weather variables on a full spatial grid. Because reanalysis synthesizes many sources of observations and utilizes simulations to produce a continuous output,  it should closely resemble observation from weather station data but should not be expected to match exactly.

* Climate models run over the historical period represent the *general conditions* during a time period, but *do not reproduce specific events from the historical record*. Each simulation from a climate model is a free-running simulation producing unique weather events and internal variability. This variability in individual climate model realizations is how we are able to determine the range of potential future realities. In the historical period, variability across each climate model realizaiton represents the range of possible conditions that could have occurred. 

**Intended Application**: As a user, I want to be able to understand the <span style="color:red">**strengths and weaknesses of comparing historical observations, reanalysis, and model output**</span> by:
1. Visualizing observations compred to reanalysis
2. Visualizing observations compared to climate model output

**Runtime**: With the default settings, this notebook takes approximately **less than 1 minute** to run from start to finish. Modifications to selections may increase the runtime. 

### Step 0: Set-up

In [1]:
import climakitae as ck
from climakitae.core.data_interface import DataParameters
import xarray as xr
import pandas as pd
import numpy as np

import hvplot.pandas  # noqa
import hvplot.xarray
pd.options.plotting.backend = 'holoviews'

### Step 1: Select data
#### 1a) Climate model data
First we will retrieve precipitation data using LOCA2-Hybrid models: multiple ensemble members for both CESM2-LENS and EC-Earth3 for a single location. 

In [2]:
selections = DataParameters()

selections.downscaling_method = 'Statistical'
selections.variable = 'Precipitation (total)'
selections.timescale = 'monthly'
selections.units = 'inches'
selections.resolution = '3 km'
selections.time_slice = (1980, 2001) # trim to the time period that overlaps between models, reanalysis, and station observations
selections.area_subset = 'lat/lon'
selections.cached_area = ['coordinate selection']
selections.area_average = 'No'
selections.latitude = (34.067 - 0.02, 34.067 + 0.02) # specifically at station coordinates, with small buffer
selections.longitude = (-117.65 - 0.02, -117.65 + 0.02) # specifically at station coordinates, with small buffer

In [3]:
# retrieving data
ds = selections.retrieve()

# subset for models of interest
mdls = ['LOCA2_EC-Earth3_r1i1p1f1', 'LOCA2_EC-Earth3_r2i1p1f1', 'LOCA2_CESM2-LENS_r1i1p1f1', 'LOCA2_CESM2-LENS_r2i1p1f1',  'LOCA2_CESM2-LENS_r3i1p1f1']
ds = ds.sel(simulation = mdls)

# loading into memory -- will take a few minutes! 
ds = ck.load(ds)
model_ds = ds.squeeze()
model_ds

Processing data to read 10.31 KB of data into memory... Complete!


<xarray.DataArray 'Precipitation (total)' (simulation: 5, time: 264)> Size: 11kB
array([[ 0.        ,  1.38207274,  3.00787196, ...,  0.        ,
         2.3123351 ,  1.71724842],
       [ 3.74225339,  9.91879759,  7.7393837 , ...,  0.        ,
         0.        ,  0.69797862],
       [ 6.95544977, 11.99669171,  2.67514393, ...,  1.04333786,
         8.60809233,  0.33705784],
       [ 0.22291127,  0.09668554,  1.02079978, ...,  0.08110002,
         0.        ,  3.58095218],
       [ 0.3388102 ,  0.        ,  1.67683313, ...,  1.42504161,
         1.59064452,  1.41613815]])
Coordinates:
    lat          float32 4B 34.08
    lon          float32 4B -117.6
  * time         (time) datetime64[ns] 2kB 1980-01-01 1980-02-01 ... 2001-12-01
  * simulation   (simulation) <U30 600B 'LOCA2_EC-Earth3_r1i1p1f1' ... 'LOCA2...
    scenario     <U18 72B 'Historical Climate'
    spatial_ref  int64 8B 0
Attributes:
    variable_id:           pr
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 inches
    data_type:             Gridded
    resolution:            3 km
    frequency:             monthly
    location_subset:       ['coordinate selection']
    approach:              Time
    downscaling_method:    Statistical
    institution:           UCSD

### 1b) Reanalysis data
Next retrieve dynamically downscaled reanalysis for the same location.

In [4]:
selections = DataParameters()

selections.variable = 'Precipitation (total)'
selections.scenario_historical = ["Historical Reconstruction"]
selections.timescale = 'monthly'
selections.units = 'inches'
selections.time_slice = (1980, 2001)
selections.area_subset = 'lat/lon'
selections.cached_area = ['coordinate selection']
selections.area_average = 'No'
selections.latitude = (34.067 - 0.03, 34.067 + 0.03) # specifically at station coordinates, with small buffer
selections.longitude = (-117.65 - 0.03, -117.65 + 0.03) # specifically at station coordinates, with small buffer

In [5]:
# retrieving data
ds = selections.retrieve()

# loading into memory -- will take a few minutes! 
ds = ck.load(ds)
reanalysis_ds = ds.squeeze()
reanalysis_ds

Processing data to read 1.00 KB of data into memory... Complete!


<xarray.DataArray 'Precipitation (total)' (time: 256)> Size: 1kB
array([4.44107413e-01, 1.25962542e-02, 1.14644421e-02, 1.04696083e+00,
       1.52545357e+00, 1.13479686e+00, 4.40542698e+00, 1.15691960e-01,
       3.55241448e-02, 4.03100885e-02, 7.64568970e-02, 9.23982784e-02,
       4.51512188e-01, 4.39686149e-01, 1.59174788e+00, 2.14391246e-01,
       5.49891615e+00, 2.09617758e+00, 6.51125479e+00, 8.59202921e-01,
       1.26744077e-01, 3.52153624e-03, 3.16782072e-02, 7.58434236e-01,
       8.98538113e-01, 1.19587235e-01, 6.09248877e+00, 2.76225948e+00,
       4.38681555e+00, 8.50817871e+00, 7.74682379e+00, 3.78112006e+00,
       9.07405391e-02, 4.11026143e-02, 1.33508397e-02, 2.25562382e+00,
       4.17821467e-01, 1.96697474e+00, 1.99585354e+00, 4.05628443e+00,
       2.21460447e-01, 1.58144742e-01, 2.80894339e-01, 4.45938855e-01,
       1.88078098e-02, 1.14579089e-01, 8.10670197e-01, 6.50713146e-01,
       3.38397831e-01, 6.49819449e-02, 1.90360069e+00, 4.70390940e+00,
       7.70002365e-01, 1.37233770e+00, 1.12078416e+00, 3.00147589e-02,
       4.82144691e-02, 4.22980869e-03, 3.05397749e-01, 8.83327723e-02,
       3.34237397e-01, 9.64599013e-01, 4.03997755e+00, 4.66929376e-01,
       1.80873227e+00, 6.38719940e+00, 4.24624872e+00, 6.32563949e-01,
       1.76617483e-04, 2.10595317e-02, 1.54210493e-01, 7.68508315e-02,
       1.10580170e+00, 2.48398289e-01, 1.50918770e+00, 5.75536311e-01,
       1.57259941e+00, 1.43774736e+00, 4.90853161e-01, 8.43009055e-02,
...
       2.93620937e-02, 1.02328509e-03, 1.22523876e-02, 5.72534800e-01,
       1.97054267e+00, 6.58012152e+00, 1.73685813e+00, 2.43230507e-01,
       4.58657891e-01, 2.32846732e-03, 7.90291131e-02, 1.26118243e-01,
       1.59231480e-02, 5.64414859e-01, 4.73032665e+00, 2.46949196e+00,
       7.11164236e+00, 1.55554339e-01, 5.06566279e-03, 3.97092737e-02,
       1.42767299e-02, 1.40607268e-01, 2.20423415e-02, 6.91729262e-02,
       1.66552842e+00, 4.70782444e-02, 1.97536623e+00, 7.38501453e+00,
       3.18763232e+00, 1.18698578e+01, 1.75030994e+00, 8.85334194e-01,
       2.27815819e+00, 1.67402133e-01, 2.66389638e-01, 4.80699927e-01,
       9.58841443e-01, 1.01681732e-01, 1.58196604e+00, 2.48038083e-01,
       1.15661335e+00, 4.01699185e-01, 5.68665624e-01, 2.21947122e+00,
       1.63785685e-02, 3.18458155e-02, 8.50164592e-02, 1.02817357e-01,
       1.40125975e-01, 4.67810128e-03, 8.63990337e-02, 8.46661907e-03,
       5.49195826e-01, 5.15795994e+00, 1.91335189e+00, 9.19170380e-01,
       1.24312574e-02, 7.32361805e-03, 2.21925266e-02, 3.44516426e-01,
       1.28729388e-01, 1.30869818e+00, 8.41416195e-02, 7.19585866e-02,
       4.19120455e+00, 9.35422230e+00, 1.45612383e+00, 7.04991519e-01,
       8.97745416e-03, 9.45444219e-03, 2.53912866e-01, 1.32851332e-01,
       2.86970064e-02, 2.27000728e-01, 2.71829271e+00, 1.38087130e+00],
      dtype=float32)
Coordinates:
    lakemask           float32 4B 0.0
    landmask           float32 4B 1.0
    lat                float32 4B 34.05
    lon                float32 4B -117.6
  * time               (time) datetime64[ns] 2kB 1980-09-01 ... 2001-12-01
    x                  float64 8B -4.071e+06
    y                  float64 8B 8.179e+05
    Lambert_Conformal  int64 8B 0
    simulation         <U8 32B 'WRF_ERA5'
    scenario           <U25 100B 'Historical Reconstruction'
Attributes:
    variable_id:           prec
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 inches
    data_type:             Gridded
    resolution:            9 km
    frequency:             monthly
    location_subset:       ['coordinate selection']
    approach:              Time
    downscaling_method:    Dynamical
    institution:           UCLA
    grid_mapping:          Lambert_Conformal

#### 1c) Observational data
Lastly, we'll read in weather observations for comparison. In this example, we are looking at precipitation observations from a weather station near Ontario, in San Bernadino County.

In [6]:
wx_obs = pd.read_csv('1026_data_cleaned.csv') ## we'll use the "total_precipitation_in" column for comparison
# trim to the time period that overlaps between models, reanalysis, and station observations
wx_obs = wx_obs[(wx_obs['year'] >= 1980) & (wx_obs['year'] <= 2001)]

# adding an easy to interpret time (month-year) column so we can compare side by side
wx_obs['day'] = 1 # using first of the month for ease
wx_obs['time'] = pd.to_datetime(wx_obs[['year', 'month', 'day']])
wx_obs = wx_obs.drop(columns=['year', 'month', 'day']) # minor cleanup

wx_obs

,WY,total_precipitation_in,days_with_data,time
360,1980,13.53,31,1980-01-01
361,1980,15.07,29,1980-02-01
362,1980,4.75,31,1980-03-01
363,1980,0.40,30,1980-04-01
364,1980,0.26,31,1980-05-01
...,...,...,...,...
619,2001,0.00,0,2001-08-01
620,2001,0.00,0,2001-09-01
621,2002,0.00,0,2001-10-01
622,2002,0.00,0,2001-11-01


### Step 2: Visualize timseries between observations, reanalysis, and model output

First, visualize the variability across realizations of climate models

In [7]:
rolling_window = 12 # use a smoothing window to better visualize trends
models_to_plot = model_ds.rolling(time=rolling_window).mean().hvplot.line(
    x='time', by='simulation', title='Inter- and Intra-model comparison', width=1000, height=400);

models_to_plot

:NdOverlay   [simulation]
   :Curve   [time]   (Precipitation (total))

Key takeaways here:
* Each run of a climate model produces a unique timeseries, with the timing wet and dry years varying from run to run. This is okay -- it's by design!
* Even within a single model, different realizations (or runs, e.g. r1i1p1f1, r2i1p1f1) produce different outputs.
* Although the timing is different in each run, the overall range of values and magnitude of interannual variability is similar across models.

Useful note: You can also "turn off" certain lines in the plot above if you want to focus on any particular model better. Just click on the name in the legend to "hide" and "unhide" a particular model.

Next, we'll plot the station observations alongside the WRF-ERA5 reanalysis product to see how these datasets compare.

In [8]:
wx_obs['total_precipitation_in_smooth'] = wx_obs['total_precipitation_in'].rolling(rolling_window).mean()
obs = wx_obs.hvplot(x='time', y='total_precipitation_in_smooth', color='black', label='Observations', 
                    line_width=3, width=1000, height=400,title='Station to Reanalysis Comparison');

models_to_plot = model_ds.sel(simulation='LOCA2_CESM2-LENS_r1i1p1f1').rolling(time=rolling_window).mean().hvplot.line(
    x='time', by='simulation', color = 'orange');
reanalysis_to_plot = reanalysis_ds.rolling(time=rolling_window).mean().hvplot.line(
    x='time', color='blue', label='ERA5 Reanalysis',  width=1000, height=400);

obs * models_to_plot * reanalysis_to_plot

:Overlay
   .Curve.Observations    :Curve   [time]   (total_precipitation_in_smooth)
   .NdOverlay.I           :NdOverlay   [simulation]
      :Curve   [time]   (Precipitation (total))
   .Curve.ERA5_Reanalysis :Curve   [time]   (Precipitation (total))

Key takeaways here:
* Unlike the climate model data (single orange line), the reanalysis (blue line) generally follows the same sequence of events as the station data (e.g. wet years in 1983 and 1993, dry period 1984-1991).
* There are still some discrepancies between the reanalysis and station observations. These could be due to limitations of the instruments at the station, calibration issues, or the reanalysis producing weather events in slightly different locations than the observations.
* An ordered timeseries from a climate model will never match the observational timeseries, but it should generally reproduce climate characteristics and variabilty (more on this below).

### Step 3: Visualize distibutions and variability over a climatology period
Next, we'll visualize the overall distribution of precipitation values to assess whether the model reproduces the *overall general conditions* of the station. We are also going to mask out the lowest 0.1 inch of precipitation from all 3 datasets. This is a common practice in climate analyses to remove *trace precipitaition* which can dramatically change the distribution of precipitation by introducing instrumentation inaccuracies to the result.

In [9]:
# Remove dry days (< 0.1 inch) to compare distribution of precipitation
model_ds = model_ds.where(model_ds > .1)
reanalysis_ds = reanalysis_ds.where(reanalysis_ds > .1)

# mask <1mm in in weather obs
valid_obs = wx_obs.loc[wx_obs['total_precipitation_in'] > .1]

In [10]:
def hist_overlay(obs, model, reanalysis):
    bins = np.arange(0,10,0.4)
    
    # relabel for legend
    obs.name = 'Observations'
    model.name = 'LOCA2_CESM2-LENS_r1i1p1f1'
    reanalysis.name = 'WRF-ERA5 reanalysis'
    
    obs_plot = obs.to_xarray().hvplot.hist(bins=bins, color='black', alpha=0.75)
    mdl_plot = model.hvplot.hist(bins=bins, color='orange')
    re_plot = reanalysis.hvplot.hist(bins=bins, color='blue', title='Observations to Model Comparison')
    return (re_plot * mdl_plot * obs_plot).options(xlabel='Precipitation (total) [inches]', 
                                                   ylabel='Number of months', width=800, height=400)

# now plot all three datasets together!
hist_overlay(obs=valid_obs['total_precipitation_in'],
             model = model_ds.sel(simulation='LOCA2_CESM2-LENS_r1i1p1f1'),
             reanalysis = reanalysis_ds)

:Overlay
   .NdOverlay.I   :NdOverlay   [Variable]
      :Histogram   [WRF-ERA5 reanalysis]   (Count)
   .NdOverlay.II  :NdOverlay   [Variable]
      :Histogram   [LOCA2_CESM2-LENS_r1i1p1f1]   (Count)
   .NdOverlay.III :NdOverlay   [Variable]
      :Histogram   [Observations]   (Count)

Now looking at the comparison of distributions, we see that the spread of monthly precipitation values lines up reasonably well between observations and model output. 
* An ordered timeseries from a climate model will never match the observational timeseries, but it should generally reproduce climate characteristics and variabilty, which we see above in the spread of values. 
* This is especially important with **precipitation**, given that precipitation does not follow a normal distribution and can be highly variable spatiotemporally and is challenging to model accurately. 

If you want to analyze real-world events from the historical record, we recommend the use of reanalysis data produccts like WRF-ERA5 (or ERA5 itself!) as an easy way to do so that is much more flexible than individual weather stations that may have gaps in their reporting record or other instrumentation inconsistencies (e.g., missing data). Climate model data will not match the year-to-year historical record, but it can be very useful to compare model output to observations or reanalysis product to evaluate how well each model represents particular features of the climate. When making these comparisons, it's important to only compare measurements aggregated over a sufficiently long enough time period (typically at least 30 years) that samples a range of the climate's natural variability. 

For more information on climate variability, check out the `internal_variability.ipynb` and `model_variability.ipynb` notebooks!